In [78]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as manim
from skimage.measure import find_contours
from sklearn.preprocessing import normalize

import pylidc as pl
from pylidc.utils import consensus
from PIL import Image, ImageDraw
import os
import matplotlib.path as mplpath

# Change deprecated functions

In [2]:
def to_matrix(self, include_k=True):
    ij = np.array([[int(cc) for cc in c.split(',')][::-1] for c in self.coords.split('\n')])
    if not include_k:
        return ij
    else:
        k  = np.ones(ij.shape[0])*self.image_k_position
        zs = self.annotation.contour_slice_zvals
        return np.c_[ij, k].astype(np.int32)

In [3]:
def boolean_mask(self, pad=None, bbox=None, include_contour_points=False):
    bb = self.bbox_matrix(pad=pad) if bbox is None else bbox

    czs = self.contour_slice_zvals
    cks = self.contour_slice_indices

    zs = self.scan.slice_zvals
    zs = zs[cks[0]:cks[-1]+1]

    z_to_index = lambda z: dict(zip(czs,cks))[z] - bb[2,0]

    ni,nj,nk = np.diff(bb, axis=1).astype(int)[:,0] + 1

    ni, nj, _ = np.amax(bb, axis=1)

    print(ni, nj)

    mask = np.zeros((ni,nj,nk), dtype=np.bool)

    ii,jj = np.indices(mask.shape[:2])
    test_points = bb[:2,0] + np.c_[ii.flatten(), jj.flatten()]

    for contour in self.contours:
        if contour.inclusion:
            zi = z_to_index(contour.image_z_position)
            C  = contour.to_matrix(include_k=False)

            if (C[0] != C[-1]).any():
                C = np.append(C, C[0].reshape(1,2), axis=0)

            path = mplpath.Path(C, closed=True)
            contains_pts = path.contains_points(test_points)
            contains_pts = contains_pts.reshape(mask.shape[:2])

            mask[:,:,zi] = np.logical_or(mask[:,:,zi], contains_pts)

            if not include_contour_points:
                i, j = (C - bb[:2,0]).T
                k = np.ones(C.shape[0], dtype=np.int32)*zi
                mask[i,j,k] = False

    for contour in self.contours:
        if not contour.inclusion:
            zi = z_to_index(contour.image_z_position)
            C = contour.to_matrix(include_k=False)

            if (C[0] != C[-1]).any():
                C = np.append(C, C[0].reshape(1,2), axis=0)

            path = mplpath.Path(C, closed=True)
            not_contains_pts = ~path.contains_points(test_points)
            not_contains_pts = not_contains_pts.reshape(mask.shape[:2])
            mask[:,:,zi] = np.logical_and(mask[:,:,zi], not_contains_pts)

            i, j = (C - bb[:2,0]).T
            k = np.ones(C.shape[0], dtype=np.int32)*zi
            mask[i,j,k] = False

    return mask

In [4]:
def consensus(anns, clevel=0.5, pad=None, ret_masks=True):
    bmats = np.array([a.bbox_matrix(pad=pad) for a in anns])
    imin,jmin,kmin = bmats[:,:,0].min(axis=0)
    imax,jmax,kmax = bmats[:,:,1].max(axis=0)

    # consensus_bbox
    cbbox = np.array([[imin,imax], [jmin,jmax], [kmin,kmax]])

    masks = [a.boolean_mask(bbox=cbbox) for a in anns]
    cbbox = tuple(slice(cb[0], cb[1]+1, None) for cb in cbbox)
    cmask = np.mean(masks, axis=0) >= clevel

    if ret_masks:
        return cmask, cbbox, masks
    else:
        return cmask, cbbox

In [88]:
def visualize_in_scan(self, verbose=True):
        images = self.scan.load_all_dicom_images(verbose)

        image_array = []
        
        # Preload contours and sort them by z pos.
        contours = sorted(self.contours, key=lambda c: c.image_z_position)
        fnames = self.scan.sorted_dicom_file_names.split(',')
        index_of_contour = [fnames.index(c.dicom_file_name) for c in contours]
        
        for current_slice in index_of_contour:

            img = images[current_slice].pixel_array
            image_array.append(img)

        contour_lines = []

        for c in contours:
            arr = c.to_matrix()
            contour_lines.append([arr[:,1], arr[:,0]])

        return image_array, contour_lines, index_of_contour

# Code

In [89]:
pl.Contour.to_matrix = to_matrix
pl.Annotation.boolean_mask = boolean_mask
pl.Annotation.visualize_in_scan = visualize_in_scan

In [146]:
global_path = ''

scanLIDC_IDRI = pl.query(pl.Scan).filter(pl.Scan.patient_id == 'LIDC-IDRI-0001')
for scan in scanLIDC_IDRI:
    path = os.path.join(global_path, scan.patient_id)
    for annotation in scan.cluster_annotations():
        for j, ann in enumerate(annotation):
            PIL_images = []
            images, contours, index = ann.visualize_in_scan()
            for image in images:
                PIL_images.append(Image.fromarray(image).convert('RGB'))
            for i, contour in enumerate(contours):
                min_x, max_x = min(contour[0]), max(contour[0])
                min_y, max_y = min(contour[1]), max(contour[1])
                shape = [(min_x, min_y), (max_x, max_y)]
                draw = ImageDraw.Draw(PIL_images[i])
                draw.rectangle(shape, outline="red", width=3)
                PIL_images[i].save(os.path.join(path, "annotation_{}_image_{}.jpg".format(j, i)), "JPEG")

Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
